In [1]:
# ------------------------------------------------------------
# 📍 0. Notebook Title & Purpose
# Employee Sentiment Analysis · Task 3: Monthly Sentiment Score
# ------------------------------------------------------------

# ------------------------------------------------------------
# 📍 1. Imports
import pandas as pd
from pathlib import Path

# ------------------------------------------------------------
# 📍 2. Load Labeled Data
df = pd.read_csv("../data/processed/labeled_messages.csv")
df["date_parsed"] = pd.to_datetime(df["date_parsed"], errors="coerce")
df = df.dropna(subset=["date_parsed"])  # Drop rows without date

# ------------------------------------------------------------
# 📍 3. Map Sentiment to Score
sentiment_map = {"Positive": 1, "Negative": -1, "Neutral": 0}
df["Sentiment_Score"] = df["Sentiment"].map(sentiment_map)

# ------------------------------------------------------------
# 📍 4. Add Year-Month Grouping
df["YearMonth"] = df["date_parsed"].dt.to_period("M")

# ------------------------------------------------------------
# 📍 5. Group by Employee and Month
monthly_scores = (
    df.groupby(["from", "YearMonth"])["Sentiment_Score"]
    .sum()
    .reset_index()
    .rename(columns={"from": "Employee", "YearMonth": "Month", "Sentiment_Score": "Score"})
)

# ------------------------------------------------------------
# 📍 6. Preview and Save
print(monthly_scores.head())
monthly_scores.to_csv("../data/processed/monthly_sentiment_scores.csv", index=False)


KeyError: 'date_parsed'